# Toxic Comment Classifier (Responsible AI Edition)

This notebook demonstrates a hybrid approach to toxic comment detection. In line with **Responsible AI** principles, we use a combination of transparent rules (keywords) and an interpretable Machine Learning model (Logistic Regression).

## 🛡️ Responsible AI Features:
- **Explainability**: Clear rules for blatant toxicity.
- **Safety**: Deterministic fallback for high-risk terms.
- **Transparency**: Interpretable model coefficients.

## 1. Setup and Imports
First, we install and import the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import os
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

## 2. Text Preprocessing
We define a robust cleaning pipeline to normalize text before it reaches the model.

In [ ]:
class TextPreprocessor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
    
    def clean_text(self, text):
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        
        # Tokenization and Lemmatization
        tokens = text.split()
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]
        
        return " ".join(tokens)

## 3. Data Generation
For this demo, we use a synthetic dataset containing toxic and non-toxic comments, including potential edge cases.

In [ ]:
def get_sample_data():
    data = [
        ("I really love the way you explained this topic. Great job!", 0),
        ("Can someone help me with this math problem?", 0),
        ("The weather today is absolutely beautiful.", 0),
        ("This movie was okay, but I've seen better.", 0),
        ("I disagree with your point, but I respect your opinion.", 0),
        ("Welcome to the community! Hope you enjoy your stay.", 0),
        ("Thank you for sharing this information.", 0),
        ("How do I install this library on Windows?", 0),
        ("The cake tastes delicious, thanks for the recipe.", 0),
        ("Good morning everyone! Have a productive day.", 0),
        ("You are such a loser, why do you even post here?", 1),
        ("I hate you and everything you stand for.", 1),
        ("Shut up, nobody cares about what you think.", 1),
        ("This is the stupidest thing I have ever read.", 1),
        ("Go away and never come back, you idiot.", 1),
        ("You're so ugly and worthless.", 1),
        ("I'm going to find you and hurt you.", 1),
        ("You are a terrible person and you should feel bad.", 1),
        ("Get out of here, we don't want people like you.", 1),
        ("You're brainless if you think this is correct.", 1),
        ("I'm going to kill this project if we don't finish it soon.", 0),
        ("This code is garbage, delete it now.", 1),
        ("You are freaking awesome!", 0),
        ("Why are you being so mean?", 0),
        ("Stop harassing people, it's not cool.", 0),
    ]
    return pd.DataFrame(data, columns=['text', 'label'])

df = get_sample_data()
df.head()

## 4. Hybrid Classifier Logic
Our classifier checks for high-risk keywords first. If none are found, it uses the ML model's prediction.

In [ ]:
class ToxicClassifier:
    def __init__(self, keyword_list=None):
        self.preprocessor = TextPreprocessor()
        self.vectorizer = TfidfVectorizer(max_features=1000)
        self.model = LogisticRegression()
        self.keyword_list = keyword_list or ['hate', 'ugly', 'stupid', 'loser', 'idiot', 'worthless', 'kill']
        self.is_trained = False

    def check_keywords(self, text):
        text_lower = text.lower()
        found_keywords = [word for word in self.keyword_list if word in text_lower]
        return len(found_keywords) > 0, found_keywords

    def train(self, df):
        print("Cleaning text...")
        df['cleaned_text'] = df['text'].apply(self.preprocessor.clean_text)
        X = self.vectorizer.fit_transform(df['cleaned_text'])
        y = df['label']
        print("Training model...")
        self.model.fit(X, y)
        self.is_trained = True

    def predict(self, text):
        keyword_flag, found = self.check_keywords(text)
        cleaned = self.preprocessor.clean_text(text)
        if not cleaned:
             ml_prob, ml_pred = 0.0, 0
        else:
            X_vec = self.vectorizer.transform([cleaned])
            ml_prob = self.model.predict_proba(X_vec)[0][1]
            ml_pred = self.model.predict(X_vec)[0]
        
        final_flag = 1 if (keyword_flag or ml_pred == 1) else 0
        return {
            'text': text,
            'keyword_flag': int(keyword_flag),
            'found_keywords': found,
            'ml_probability': float(ml_prob),
            'final_prediction': final_flag
        }

## 5. Training and Evaluation
We train the model and visualize its performance using a Confusion Matrix.

In [ ]:
classifier = ToxicClassifier()
classifier.train(df)

# Evaluation
y_true = df['label']
y_pred = classifier.model.predict(classifier.vectorizer.transform(df['text'].apply(classifier.preprocessor.clean_text)))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Toxic', 'Toxic']))

# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

## 6. Interactive Testing
Test the hybrid logic with your own comments!

In [ ]:
test_comments = [
    "You are an absolute genius!",
    "This is stupid code.",
    "I'm going to kill this bug.",
    "You are a worthless loser."
]

for comment in test_comments:
    res = classifier.predict(comment)
    status = "🚩 TOXIC" if res['final_prediction'] == 1 else "✅ SAFE"
    print(f"Comment: {comment}")
    print(f"Result: {status} (ML Prob: {res['ml_probability']:.2f}, Keywords: {res['found_keywords']})\n")